In [2]:
#Transfer Matrix Method for CROW containing 2 rings.
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
k_pairs = [
    (-0.2j, -0.1j),
    (-0.1j, -0.2j),
    (-0.3j, -0.6j),
    (-0.6j, -0.8j),
    (-0.7j, -0.5j),
    (-0.4j, -0.9j),
    (-0.2j, -1.0j),
    (-1.0j, -0.2j)
]  #first entry is k_ring other one is k_out/k_in

def compute_transmission(k, k_out):
    
    t = np.sqrt(1 - abs(k)**2)
    t_in  = np.sqrt(1 - abs(k_out)**2)
    
    c = 3e8  # speed of light (m/s)
    R = 164.5e-6  # ring radius in meters
    L = 2 * np.pi * R  # ring circumference
    m = 100  # integer resonance mode number (for 1.5 µm)
    
    lambdas = np.linspace(1.549e-6, 1.551e-6, 500)  # wavelength sweep in meters
    omegas = 2 * np.pi * c / lambdas  # angular frequency
    
    # Defining the P matrices
    
    P = (1/k)*np.array([[-t, 1], [-1, np.conj(t)]])
    P_in = (1/k_out)*np.array([[-t_in, 1], [-1, np.conj(t_in)]])
    P_out = P_in #**
    
    #The matrix Q will be defined inside the loop while defining the transfer matrices for each ring as it is frequency dependent, where frequency is set by the input signal.
    #Two types of transfer matrices - through and output ports
    #Defining two empty arrays for through and drop port transmittances
    t_thr = []
    t_drop = []
    p_thr = []
    p_drop = []
    N =  2#number of rings in the CROWs
    n_eff = 1.5  #effective refractive index - r.i. at a particular frequency (otherwise it is also frequency dependent)
    
     #defining the wavelength range for the input signal. Resolving into 1000 points.
    
    for omega in omegas:
        beta = ((n_eff * omega)/ c) #no imaginary component
        phase = beta * R * np.pi
        Q = np.array([[0, np.exp(-1j * phase)],[np.exp(1j * phase), 0]])
       # print(Q)
        #now P and Q need to multiplied N-1 times for which we must define an identity matrix to initialise their product.
        I = np.eye(2) #a 2x2 identity matrix
        for _ in range(N-1):
            I = P @ Q @ I
    
        #total transfer matrix
        T = P_out @ Q @ I @ P_in
    
        #separating matrix components
        A = T[0,0]
        B = T[0,1]
        C = T[1,0]
        D = T[1,1]
        #print (T)
        T_thr = -A/B
        T_out = C - ((A*D)/B)
    
        phase_thr = np.angle(T_thr)
        phase_out = np.angle(T_out)
    
        #unwrap not working!!
    
       # print(A,B,C,D)
      #  print(T_thr)
    
        #since these are complex values, in order to plot them we will store these as their absolute values.
        t_thr.append(abs(T_thr)**2)
        t_drop.append(abs(T_out)**2)
    
        p_thr.append(phase_thr)
        p_drop.append(phase_out)
        
    return t_thr, t_drop, omegas

In [4]:
#to clearly format the file name

def format_complex(z):
    # Returns a string like "0.20j" or "0.10j" or "0.10+0.20j"
    if np.isreal(z):
        return f"{z.real:.2f}"
    elif z.real == 0:
        return f"{z.imag:.2f}j"
    elif z.imag == 0:
        return f"{z.real:.2f}"
    else:
        return f"{z.real:.2f}+{z.imag:.2f}j"

In [5]:
def plot_transmissions(T1, T2, freq, k, k_out):
    plt.figure(figsize=(10, 5))
    # plt.plot(freq, T1, label="Through Port", lw=2)
    plt.plot(freq, T2, label="Drop Port", lw=2)
    plt.title(f"Transmission Plot (k = {format_complex(k)}, k_out = {format_complex(k_out)})")
    plt.xlabel("Frequency")
    plt.ylabel("Transmission")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    filename = f"droptrans_k_{format_complex(k)}_kout_{format_complex(k_out)}.png"
    plt.savefig(filename, dpi=300)
    plt.close()

In [6]:
for k, k_out in k_pairs:
    T1, T2, omegas = compute_transmission(k, k_out)
    plot_transmissions(T1, T2, omegas, k, k_out)